In [5]:
import pandas as pd
import sqlite3
from config import cmconf
from pathlib import Path

# 데이터베이스 불러오기

In [7]:
dbpath = Path(cmconf['path']['root']) / Path(cmconf['path']['db'])
conn = sqlite3.connect(dbpath)
cur = conn.cursor()

# 기업과 관련된 재무제표 불러오기

In [101]:
corp_name = '3S'

In [94]:
sql = '''
select c.name as 'corp_name', m.name as 'market', r.year, r.quarter, rt.name, r.date, ri.account_code, ri.account_name, ri.amount
from Report as r
inner join ReportItem as ri on ri.report = r.id
inner join StockCode as sc on sc.corporation = r.corporation
inner join corporation as c on c.id = r.corporation
inner join market as m on sc.market = m.id
inner join sector as st on c.sector = st.id
inner join ReportType as rt on rt.id = r.report_type
where c.name = ?
'''
# columns = [
#     '회사명', '시장', '연도', '분기', '보고서유형', '결산일', '계정코드', '계정명', '금액'
# ]
df = pd.read_sql(sql, conn, params=(corp_name,))

In [95]:
df

,corp_name,market,year,quarter,name,date,account_code,account_name,amount
0,3S,코스닥시장상장법인,2017,반기보고서,"재무상태표, 유동/비유동법-별도재무제표",2017-09-30,ifrs_AssetsAbstract,자산 [abstract],nan
1,3S,코스닥시장상장법인,2017,반기보고서,"재무상태표, 유동/비유동법-별도재무제표",2017-09-30,ifrs_CurrentAssets,유동자산,11706322326
2,3S,코스닥시장상장법인,2017,반기보고서,"재무상태표, 유동/비유동법-별도재무제표",2017-09-30,ifrs_CashAndCashEquivalents,현금및현금성자산,2546144443
3,3S,코스닥시장상장법인,2017,반기보고서,"재무상태표, 유동/비유동법-별도재무제표",2017-09-30,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,984355188
4,3S,코스닥시장상장법인,2017,반기보고서,"재무상태표, 유동/비유동법-별도재무제표",2017-09-30,ifrs_TradeAndOtherCurrentReceivables,매출채권및기타채권,5587474227
5,3S,코스닥시장상장법인,2017,반기보고서,"재무상태표, 유동/비유동법-별도재무제표",2017-09-30,dart_ShortTermDueFromCustomersForContractWork,미청구공사,1335933237
6,3S,코스닥시장상장법인,2017,반기보고서,"재무상태표, 유동/비유동법-별도재무제표",2017-09-30,ifrs_Inventories,재고자산,909341820
7,3S,코스닥시장상장법인,2017,반기보고서,"재무상태표, 유동/비유동법-별도재무제표",2017-09-30,ifrs_CurrentTaxAssets,당기법인세자산,2445030
8,3S,코스닥시장상장법인,2017,반기보고서,"재무상태표, 유동/비유동법-별도재무제표",2017-09-30,dart_OtherCurrentAssets,기타유동자산,235748621
9,3S,코스닥시장상장법인,2017,반기보고서,"재무상태표, 유동/비유동법-별도재무제표",2017-09-30,dart_CurrentDerivativeAsset,파생금융상품,104879760


## 의미있는 문자열로 변환

In [96]:
df['quarter'] = df['quarter'].str.replace('반기보고서', '2Q')

## 관심있는 계정만 추출

In [97]:
df2 = df[df['account_code'].isin(['ifrs_Assets', 'ifrs_Equity', 'ifrs_Liabilities'])]

## 금액 변환(억 단위)

In [98]:
df2.loc[:, 'amount'] = df2['amount'].apply(lambda x: int(x) / 100000000)

In [99]:
df2

,corp_name,market,year,quarter,name,date,account_code,account_name,amount
18,3S,코스닥시장상장법인,2017,2Q,"재무상태표, 유동/비유동법-별도재무제표",2017-09-30,ifrs_Assets,자산총계,490.54557
31,3S,코스닥시장상장법인,2017,2Q,"재무상태표, 유동/비유동법-별도재무제표",2017-09-30,ifrs_Liabilities,부채총계,215.818107
37,3S,코스닥시장상장법인,2017,2Q,"재무상태표, 유동/비유동법-별도재무제표",2017-09-30,ifrs_Equity,자본총계,274.727462


In [100]:
# 자산대비 부채비율
100 * df2.loc[df2['account_code'] == 'ifrs_Liabilities', 'amount'] / df2.loc[df2['account_code'] == 'ifrs_Assets', 'amount'].values

31    43.995527
Name: amount, dtype: object

# 재무제표에서 해야할 질문
## 매출, 영업이익, 비용, 부채

   기업가치 = f(매출, 영업이익, 비용, 부채)

### 매출이 기간동안 증가하고 있는가?, 얼만큼 증가하는가?

### 영업이익이 기간동안 증가하고 있는가?, 얼만큼 증가하는가?

### 비용이 기간동안 감소하거나 정체하고 있는가?

### 부채가 기간동안 감소하거나 정체하고 있는가?

## 최근 4분기(1년) 동안 매출 증가 비율

# Macro 분석 지표
## 기준 금리
### 국채 시장 금리
### 회사채 시장 금리
## 경제성장률